This notebook plots the HSC luminosity functions (from https://arxiv.org/abs/2108.01090), and calculates numbers for LSST.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import Planck18
from astropy import units as u
from photerr import LsstErrorModel

#### Parameters for plotting luminosity functions

In [ ]:
# Schechter params from GOLDRUSH IV
sch_params = {
    3 : {
        "norm": -2.84,
        "Mstar": -20.91,
        "alpha": -1.68,
    },
    4 : {
        "norm": -2.69,
        "Mstar": -20.72,
        "alpha": -1.68,
    },
    5 : {
        "norm": -3.10,
        "Mstar": -21.04,
        "alpha": -1.76,
    },
    6 : {
        "norm": -3.28,
        "Mstar": -20.90,
        "alpha": -1.97,
    },
    7 : {
        "norm": -3.17,
        "Mstar": -20.54,
        "alpha": -1.89,
    },
}

# Schechter params from Bouwens 2021
hst_sch_params = {
    4 : {
        "norm": -2.77,
        "Mstar": -20.93,
        "alpha": -1.69,
    },
    5 : {
        "norm": -3.10,
        "Mstar": -21.10,
        "alpha": -1.74,
    },
    6 : {
        "norm": -3.29,
        "Mstar": -20.93,
        "alpha": -1.93,
    },
    7 : {
        "norm": -3.72,
        "Mstar": -21.15,
        "alpha": -2.06,
    },
    8 : {
        "norm": -4.05,
        "Mstar": -20.93,
        "alpha": -2.23,
    },
    9 : {
        "norm": -4.68,
        "Mstar": -21.15,
        "alpha": -2.33,
    },
    10 : {
        "norm": -5.38,
        "Mstar": -21.19,
        "alpha": -2.38,
    },
}

def sch_lf(M, params):
    """A Schechter Luminosity function"""
    norm = np.log(10) / 2.5 * (10 ** params["norm"])
    pl = 10 ** (-(1 + params["alpha"]) / 2.5 * (M - params["Mstar"]))
    exp = np.exp(-10 ** (-1/2.5 * (M - params["Mstar"])))
    return norm * pl * exp

In [ ]:
dpl_params = {
    3: {
        "norm": -3.23,
        "Mstar": -21.30,
        "alpha": -1.89,
        "beta": -4.78,
    },
    4: {
        "norm": -3.09,
        "Mstar": -21.10,
        "alpha": -1.87,
        "beta": -4.95,
    },
    5: {
        "norm": -3.48,
        "Mstar": -21.39,
        "alpha": -1.94,
        "beta": -4.96,
    },
    6: {
        "norm": -3.67,
        "Mstar": -21.23,
        "alpha": -2.14,
        "beta": -5.03,
    },
    7: {
        "norm": -3.51,
        "Mstar": -20.82,
        "alpha": -2.05,
        "beta": -4.83,
    },
}

def dpl_lf(M, params):
    """A Double Power Law Luminosity function"""
    norm = np.log(10) / 2.5 * (10 ** params["norm"])
    dpl = (
        10 ** ((1 + params["alpha"]) / 2.5 * (M - params["Mstar"])) + 
        10 ** ((1 + params["beta"]) / 2.5 * (M - params["Mstar"]))
    )
    return norm / dpl

#### LF observations from HSC and HST

In [ ]:
hsc = {
    3: {
        "mags": [],
        "lf": [],
        "err+": [],
        "err-": [],
    },
    4: {
        "mags": [-24.39, -24.09, -23.79, -23.49, -23.19, -22.89, -22.64, -22.44, -22.29, -22.19, -22.09, -21.99, -21.89, -21.79, -21.69, -21.59, -21.49, -21.39, -21.29, -21.19, -21.09, -20.99, -20.89, -20.79, -20.69, -20.59, -20.49, -20.39, -20.29, -20.19, -20.09, -19.99],
        "lf": [1.11e-8, 4.30e-8, 9.22e-8, 2.17e-7, 3.63e-7, 8.07e-7, 1.97e-6, 5.67e-6, 9.24e-6, 1.53e-5, 1.89e-5, 2.75e-5, 3.66e-5, 5.44e-5, 7.38e-5, 1.03e-4, 1.28e-4, 1.82e-4, 2.23e-4, 2.92e-4, 3.86e-4, 4.58e-4, 5.16e-4, 6.35e-4, 8.15e-4, 9.08e-4, 1.03e-3, 1.19e-3, 1.46e-3, 1.59e-3, 1.78e-3, 2.02e-3],
        "err+": [2.74e-8, 5.65e-8, 9.32e-8, 1.83e-7, 2.30e-7, 4.28e-7, 0.92e-6, 2.01e-6, 4.10e-6, 0.61e-5, 0.68e-5, 0.92e-5, 1.13e-5, 1.58e-5, 1.89e-5, 0.23e-4, 0.26e-4, 0.34e-4, 0.38e-4, 0.49e-4, 0.67e-4, 0.81e-4, 0.93e-4, 1.16e-4, 1.51e-4, 1.72e-4, 0.20e-3, 0.24e-3, 0.31e-3, 0.36e-3, 0.44e-3, 0.54e-3],
        "err-": [1.11e-8, 4.30e-8, 9.22e-8, 1.83e-7, 2.30e-7, 4.28e-7, 0.92e-6, 2.01e-6, 4.10e-6, 0.61e-5, 0.68e-5, 0.92e-5, 1.13e-5, 1.58e-5, 1.89e-5, 0.23e-4, 0.26e-4, 0.34e-4, 0.38e-4, 0.49e-4, 0.67e-4, 0.81e-4, 0.93e-4, 1.16e-4, 1.51e-4, 1.72e-4, 0.20e-3, 0.24e-3, 0.31e-3, 0.36e-3, 0.44e-3, 0.54e-3],
    },
    5: {
        "mags": [-24.42, -23.92, -23.42, -23.05, -22.80, -22.55, -22.30, -22.05, -21.80, -21.55, -21.30, -21.05, -20.80, -20.55, -20.30],
        "lf": [2.58e-9, 4.73e-8, 1.57e-7, 6.22e-7, 1.27e-6, 5.21e-6, 1.24e-5, 2.97e-5, 4.78e-5, 1.00e-4, 1.77e-4, 2.93e-4, 4.09e-4, 4.31e-4, 6.13e-4],
        "err+": [14.68e-9, 10.22e-8, 2.48e-7, 4.19e-7, 0.67e-6, 2.28e-6, 0.43e-5, 0.80e-5, 1.10e-5, 0.20e-4, 0.38e-4, 0.71e-4, 1.14e-4, 1.50e-4, 2.52e-4],
        "err-": [2.58e-9, 4.73e-8, 1.57e-7, 4.19e-7, 0.66e-6, 2.28e-6, 0.43e-5, 0.80e-5, 1.10e-5, 0.20e-4, 0.38e-4, 0.71e-4, 1.14e-4, 1.50e-4, 2.52e-4],
    },
    6: {
        "mags": [-24.02, -23.52, -23.12, -22.82, -22.52, -22.22, -21.92, -21.62, -21.32, -21.02],
        "lf": [6.00e-9, 3.76e-8, 1.80e-7, 7.59e-7, 1.81e-6, 3.46e-6, 9.58e-6, 3.55e-5, 7.35e-5, 1.77e-4],
        "err+": [9.95e-9, 2.97e-8, 1.09e-7, 4.14e-7, 0.96e-6, 1.78e-6, 4.82e-6, 1.80e-5, 4.05e-5, 1.22e-4],
        "err-": [6.00e-9, 2.05e-8, 1.09e-7, 4.14e-7, 0.96e-6, 1.78e-6, 4.82e-6, 1.80e-5, 4.05e-5, 1.22e-4],
    },
    7: {
        "mags": [-24.42, -23.92, -23.42, -22.92, -22.42, -21.92],
        "lf": [5.00e-10, 1.31e-8, 4.39e-8, 1.83e-7, 1.06e-6, 2.75e-6],
        "err+": [24.81e-10, 2.46e-8, 6.02e-8, 3.62e-7, 1.20e-6, 2.77e-6],
        "err-": [5.00e-10, 1.31e-8, 4.39e-8, 1.83e-7, 1.06e-6, 2.55e-6],
    },
}

In [ ]:
hubble = {
    3: {
        "mags": [-22.52, -21.77, -21.27, -20.77, -20.27, -19.77, -19.27, -18.77, -18.27, -17.77, -17.27],
        "lf": [6e-6, 7.6e-5, 4.02e-4, 7.69e-4, 1.607e-3, 2.205e-3, 3.521e-3, 4.557e-3, 6.258e-3, 1.1417e-2, 1.0281e-2],
        "err+": [5e-6, 3.8e-5, 7.8e-5, 1.17e-4, 1.57e-4, 1.89e-4, 2.39e-4, 2.97e-4, 4.37e-4, 6.56e-4, 1.368e-3],
        "err-": [5e-6, 3.8e-5, 7.8e-5, 1.17e-4, 1.57e-4, 1.89e-4, 2.39e-4, 2.97e-4, 4.37e-4, 6.56e-4, 1.368e-3],
    },
    4: {
        "mags": [-22.69, -22.19, -21.69, -21.19, -20.69, -20.19, -19.69, -19.19, -18.69, -17.94, -16.94, -15.94],
        "lf": [5e-6, 1.5e-5, 1.44e-4, 3.44e-4, 6.98e-4, 1.624e-3, 2.276e-3, 3.056e-3, 4.371e-3, 1.0160e-2, 2.7420e-2, 2.8820e-2],
        "err+": [4e-6, 9e-6, 2.2e-5, 3.8e-5, 6.8e-5, 1.31e-4, 1.99e-4, 3.88e-4, 6.89e-4, 9.20e-4, 3.440e-3, 8.740e-3],
        "err-": [4e-6, 9e-6, 2.2e-5, 3.8e-5, 6.8e-5, 1.31e-4, 1.99e-4, 3.88e-4, 6.89e-4, 9.20e-4, 3.440e-3, 8.740e-3],
    },
    5: {
        "mags": [-23.11, -22.61, -22.11, -21.61, -21.11, -20.61, -20.11, -19.61, -19.11, -18.36, -17.36, -16.36],
        "lf": [1e-6, 4e-6, 2.8e-5, 9.2e-5, 2.62e-4, 5.84e-4, 8.79e-4, 1.594e-3, 2.159e-3, 4.620e-3, 8.780e-3, 2.5120e-2],
        "err+": [1e-6, 2e-6, 7e-6, 1.3e-5, 2.4e-5, 4.4e-5, 6.7e-5, 1.56e-4, 3.46e-4, 5.20e-4, 1.540e-3, 7.340e-3],
        "err-": [1e-6, 2e-6, 7e-6, 1.3e-5, 2.4e-5, 4.4e-5, 6.7e-5, 1.56e-4, 3.46e-4, 5.20e-4, 1.540e-3, 7.340e-3],
    },
    6: {
        "mags": [-22.52, -22.02, -21.52, -21.02, -20.52, -20.02, -19.52, -18.77, -17.77, -16.77],
        "lf": [2e-6, 1.4e-5, 5.1e-5, 1.69e-4, 3.17e-4, 7.24e-4, 1.147e-3, 2.820e-3, 8.360e-3, 1.7100e-2],
        "err+": [2e-6, 5e-6, 1.1e-5, 2.4e-5, 4.1e-5, 8.7e-5, 1.57e-4, 4.40e-4, 1.660e-3, 5.260e-3],
        "err-": [2e-6, 5e-6, 1.1e-5, 2.4e-5, 4.1e-5, 8.7e-5, 1.57e-4, 4.40e-4, 1.660e-3, 5.260e-3],
    },
    7: {
        "mags": [-22.19, -21.69, -21.19, -20.69, -20.19, -19.69, -19.19, -18.69, -17.94, -16.94],
        "lf": [1e-6, 4.1e-5, 4.7e-5, 1.98e-4, 2.83e-4, 5.89e-4, 1.172e-3, 1.433e-3, 5.760e-3, 8.320e-3],
        "err+": [2e-6, 1.1e-5, 1.5e-5, 3.6e-5, 6.6e-5, 1.26e-4, 3.36e-4, 4.19e-4, 1.440e-3, 2.900e-3],
        "err-": [2e-6, 1.1e-5, 1.5e-5, 3.6e-5, 6.6e-5, 1.26e-4, 3.36e-4, 4.19e-4, 1.440e-3, 2.900e-3],
    },
}

#### Function to plot LFs along with the data

In [ ]:
def plot_lf(z, sch=True, dpl=False, data=False, lsstComm=False, lsstFull=False):
    """Plot the luminosity function for the given redshift.

    Parameters
    ----------
    z: int
        The redshift for which to plot the luminosity function. 
        Can be one of [3, 4, 5, 6, 7].
    sch: bool, default=True
        Whether to plot the Schechter Luminosity function
    dpl: bool, default=False
        Whether to plot the DPL Luminosity function
    data: bool, default=False
        Whether to plot the Hubble and HSC data
    lsstComm: bool, default=False
        Whether to plot the forecast for LSST Commissioning
    lsstFull: bool, default=False
        Whether to plot the forecast for the full LSST survey

    Returns
    -------
    plt.Figure
        The matplotlib figure
    plt.Axes
        The matplotlib axis
    """
    # create the figure
    fig, ax = plt.subplots(figsize=(4.5, 3.3), dpi=120)

    # plot the luminosity function
    # using parameters from GOLDRUSH IV
    M = np.linspace(-27, -15, 100_000)
    if sch:
        ax.plot(M, sch_lf(M, sch_params[z]), c="gray", label="Luminosity function")
    if dpl:
        ax.plot(M, sch_lf(M, dpl_params[z]), c="gray", label="Luminosity function")

    if data:
        # plot the HSC data
        ax.errorbar(
            hsc[z]["mags"], 
            hsc[z]["lf"], 
            yerr=[hsc[z]["err-"], hsc[z]["err+"]], 
            ls="", 
            marker="o", 
            markersize=4,
            capsize=3, 
            c="C0", 
            label="HSC (Harikane 2022)", 
        )

        # plot Hubble data
        ax.errorbar(
            hubble[z]["mags"], 
            hubble[z]["lf"], 
            yerr=[hubble[z]["err-"], hubble[z]["err+"]], 
            ls="", 
            marker="s", 
            markersize=4,
            capsize=3, 
            c="k", 
            label="Hubble (Bouwens 2021)", 
        )

    if lsstComm or lsstFull:
        # simulate LSST observations
        # --------------------------

        # determine the width and depths
        if lsstComm:
            wide_time = 2 # years
            wide_area = 1000 # sq deg
            deep_time = 20 # years
            deep_area = 100 # sq deg
        elif lsstFull:
            wide_time = 10 # years
            wide_area = 18_000 # sq deg
            deep_time = 200 # years
            deep_area = 40 # sq deg

        bin_width = 0.25
        bin_centers = np.arange(-27, -15, bin_width) + bin_width / 2

        # total volume (Mpc^3)
        tot_vol = Planck18.comoving_volume(z + 0.5) - Planck18.comoving_volume(z - 0.5)
        tot_vol = tot_vol.value

        # fraction of the sky
        sky_frac = (wide_area * u.deg**2).to(u.steradian) / (4 * np.pi * u.steradian)

        # observed volume
        obs_vol = sky_frac * tot_vol

        # number of galaxies observed
        binned_lf = dpl_lf(bin_centers, dpl_params[z])
        N_galaxies = binned_lf * obs_vol * bin_width

        # above the deep limit, we do not detect any galaxies
        detected_band = {3: "u", 4: "g", 5: "r", 6: "i", 7: "z"}[z]
        deep_model = LsstErrorModel(nYrObs=deep_time)
        deep_limit = (
            deep_model.getLimitingMags()[detected_band]
            + 2.5 * np.log10(1 + z)
            - 5 * np.log10(Planck18.luminosity_distance(z) / (10 * u.pc))
        )
        N_galaxies[bin_centers > deep_limit] *= 0

        # sample from Poisson
        rng = np.random.default_rng(4)
        N_obs = rng.poisson(N_galaxies)

        # calculate observations
        lf_obs = N_obs / obs_vol / bin_width
        errs = np.sqrt(N_obs) / obs_vol / bin_width

        # above the wide limit, we need to use the deep field, which has 10x less area
        # so we magnify the Poisson errors by sqrt(10)
        wide_model = LsstErrorModel(nYrObs=wide_time)
        wide_limit = (
            wide_model.getLimitingMags()[detected_band]
            + 2.5 * np.log10(1 + z)
            - 5 * np.log10(Planck18.luminosity_distance(z) / (10 * u.pc))
        )
        errs[bin_centers > wide_limit] *= np.sqrt(wide_area / deep_area)

        ax.errorbar(
            bin_centers, 
            lf_obs, 
            yerr=errs, 
            ls="", 
            marker="*", 
            markersize=6,
            capsize=3, 
            c="C3", 
            label="LSST Commissioning",
        )

        # before we're done, let's print the total number of galaxies!
        # we need to down-weight the number of galaxies in the deep field (when I
        # corrected for the deep field above, I only scaled the error bars)
        N_obs = N_obs.astype(float)
        N_obs[bin_centers > wide_limit] /= wide_area / deep_area

        # finally, let's print the number of galaxies!
        print(f"total galaxies: {N_obs.sum() / 1e6: .2f}e6")

    ax.legend(loc="lower right", fontsize=9)
    ax.set(
        yscale="log", 
        xlabel="$M_{UV}$", 
        ylabel="$\phi(M_{UV})$  [Mpc$^{-3}$ mag$^{-1}$ ]",
        xlim=(-26, -15.5),
        ylim=(1e-11, 1e-1),
    )
    ax.text(0.05, 0.95, f"z ~ {z}", transform=ax.transAxes, va="top", ha="left")

    plt.tight_layout()

    return fig, ax

### plot Luminosity Function for the introduction

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4.5, 3.3), dpi=120)
ax.set(
    yscale="log", 
    xlabel="$M_{UV}$", 
    ylabel="$\phi(M_{UV})$  [Mpc$^{-3}$ mag$^{-1}$ ]",
    xlim=(-24, -16),
    ylim=(1e-10, 1e-1),
)

# plot a single luminosity function
M = np.linspace(-27, -15, 100_000)
ax.plot(M, sch_lf(M, hst_sch_params[4]), label="z ~ 4", c="C0")
ax.legend(loc="lower right")

plt.tight_layout()
fig.savefig("figures/lf_intro.png", dpi=500)

# plot the evolution of the luminosity function
ax.plot(M, sch_lf(M, hst_sch_params[6]), label="z ~ 6", c="C2")
ax.plot(M, sch_lf(M, hst_sch_params[8]), label="z ~ 8", c="C1")
ax.plot(M, sch_lf(M, hst_sch_params[10]), label="z ~ 10", c="C3")
ax.legend(loc="lower right")

fig.savefig("figures/lf_evolution.png", dpi=500)

### Bright end excess

Note I moved this to the bottom of the notebook where I plot the lensed LF as well

In [ ]:
fig, ax = plot_lf(4, data=True)
ax.set(xlim=(-25, -15.5))
#fig.savefig("figures/lf_bright_excess.png", dpi=500)

### Commissioning forecast

Note I moved this to the bottom of the notebook where I plot the lensed LF as well

In [ ]:
fig, ax = plot_lf(4, data=True, lsstComm=False)
#fig.savefig("figures/lf_before_commissioning.png", dpi=500)

In [ ]:
fig, ax = plot_lf(4, data=True, lsstComm=True)
#fig.savefig("figures/lf_after_commissioning.png", dpi=500)

### Number of galaxies per redshift bin

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 3.3), dpi=120)

# numbers printed with the plots below
z = [3, 4, 5, 6, 7]
N_before = [786818, 1843484, 142808, 3633, 893]
N_lsst = [9.79e6, 25.09e6, 10.28e6, 1.80e6, 0.12e6]

# create the stacked bar plot
lsst_bars = ax.bar(z, N_lsst, log=False, color="C3", label="LSST Commissioning")
ax.bar(z, N_before, log=False, color="C0", label="HSC + Hubble")
ax.legend(fontsize=9)

# add bar labels
exp = np.floor(np.log10(lsst_bars.datavalues))
coeff = lsst_bars.datavalues / 10 ** exp
ax.bar_label(
    lsst_bars, 
    labels=[f"${c:.1f}\\times10^{int(e)}$" for c, e in zip(coeff, exp)],
)

ax.set(ylim=(0, 2.8e7), xlabel="Redshift", ylabel="Number of galaxies")
plt.tight_layout()

fig.savefig(f"figures/Ngalaxies_lsstComm.png", dpi=600)

In [ ]:
exp = np.floor(np.log10(lsst_bars.datavalues))
coeff = lsst_bars.datavalues / 10 ** exp

In [ ]:
lsst_bars.datavalues

### Backup Commissioning slides

In [ ]:
z = 3
fig, ax = plot_lf(z, data=True, lsstComm=True)
fig.savefig(f"figures/lf_z={z}_lsstComm.png", dpi=500)

In [ ]:
z = 4
fig, ax = plot_lf(z, data=True, lsstComm=True)
fig.savefig(f"figures/lf_z={z}_lsstComm.png", dpi=500)

In [ ]:
z = 5
fig, ax = plot_lf(z, data=True, lsstComm=True)
fig.savefig(f"figures/lf_z={z}_lsstComm.png", dpi=500)

In [ ]:
z = 6
fig, ax = plot_lf(z, data=True, lsstComm=True)
fig.savefig(f"figures/lf_z={z}_lsstComm.png", dpi=500)

In [ ]:
z = 7
fig, ax = plot_lf(z, data=True, lsstComm=True)
fig.savefig(f"figures/lf_z={z}_lsstComm.png", dpi=500)

### Backup LSST Slides

In [ ]:
z = 3
fig, ax = plot_lf(z, data=True, lsstFull=True)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig(f"figures/lf_z={z}_lsstFull.png", dpi=500)

In [ ]:
z = 4
fig, ax = plot_lf(z, data=True, lsstFull=True)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig(f"figures/lf_z={z}_lsstFull.png", dpi=500)

In [ ]:
z = 5
fig, ax = plot_lf(z, data=True, lsstFull=True)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig(f"figures/lf_z={z}_lsstFull.png", dpi=500)

In [ ]:
z = 6
fig, ax = plot_lf(z, data=True, lsstFull=True)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig(f"figures/lf_z={z}_lsstFull.png", dpi=500)

In [ ]:
z = 7
fig, ax = plot_lf(z, data=True, lsstFull=True)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig(f"figures/lf_z={z}_lsstFull.png", dpi=500)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 3.3), dpi=120)

# numbers printed with the plots above
z = [3, 4, 5, 6, 7]
N_before = [786818, 1843484, 142808, 3633, 893]
N_lsst = [482.25e6, 1057.05e6, 343.78e6, 109.38e6, 9.63e6]

# create the stacked bar plot
ax.bar(z, N_lsst, log=True, color="C3", label="LSST Commissioning")
ax.bar(z, N_before, log=True, color="C0", label="HSC + Hubble")
ax.legend(fontsize=9)

ax.set(ylim=(1e2, 2e10), xlabel="Redshift", ylabel="Number of galaxies")
plt.tight_layout()

fig.savefig(f"figures/Ngalaxies_lsstFull.png", dpi=600)

### Impacts of assuming mean redshift and no K corrections, plus simulating lensing magnification

In [ ]:
z = 4

wide_time = 10 # years
wide_area = 18_000 # sq deg
deep_time = 200 # years
deep_area = 40 # sq deg

bin_width = 0.25
bins = np.arange(-27, -19 + bin_width, bin_width)
bin_centers = (bins[:-1] + bins[1:]) / 2

# total volume (Mpc^3)
tot_vol = Planck18.comoving_volume(z + 0.5) - Planck18.comoving_volume(z - 0.5)
tot_vol = tot_vol.value

# fraction of the sky
sky_frac = (wide_area * u.deg**2).to(u.steradian) / (4 * np.pi * u.steradian)
sky_frac = sky_frac.value

# observed volume
obs_vol = sky_frac * tot_vol

# number of galaxies observed
binned_lf = sch_lf(bin_centers, sch_params[z])
N_galaxies = np.sum(binned_lf * obs_vol * bin_width).astype(int)

# we will simulate this number of galaxies
rng = np.random.default_rng(0)

# first sample the true magnitudes
M = np.linspace(bins.min(), bins.max(), 100_000)
lf = sch_lf(M, sch_params[z])
lf /= lf.sum()
Mtrue = rng.choice(M, N_galaxies, p=lf)

# now sample the true redshifts
# this Gaussian is taken from the z~4 approximation in redshift_distribution.ipynb
ztrue = rng.normal(4, 0.42, N_galaxies)

First, the error from assuming that all redshifts are $\bar{z}$

In [ ]:
# calculate the absolute magnitude adjustment as a function of redshift
zgrid = np.linspace(ztrue.min() - 0.1, ztrue.max() + 0.1, 100_00)
dMgrid = - 2.5 * np.log10((1 + zgrid)/(1 + z)) + 5 * np.log10(Planck18.luminosity_distance(zgrid).value/Planck18.luminosity_distance(z).value)
dM = lambda z: np.interp(z, zgrid, dMgrid)

# calculate the observed absolute magnitudes
Mobs_zerr = Mtrue + dM(ztrue)

In [ ]:
# bin the observed magnitudes
weights_zerr, _ = np.histogram(Mobs_zerr, bins=bins)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 3.3), dpi=120)

# plot the LF
ax.plot(M, sch_lf(M, sch_params[z]), c="gray", label="Luminosity function")

# plot the observed LF
binned_lf = weights_zerr / obs_vol / bin_width
errs = np.sqrt(weights_zerr) / obs_vol / bin_width
ax.errorbar(
    bin_centers, 
    binned_lf, 
    yerr=errs, 
    ls="", 
    marker=".", 
    markersize=4,
    capsize=3, 
    c="C3",
    label="Inferred using $\\bar{z}$"
)

ax.legend(loc="lower right", fontsize=9)
ax.set(
    yscale="log", 
    xlabel="$M_{UV}$", 
    ylabel="$\phi(M_{UV})$  [Mpc$^{-3}$ mag$^{-1}$ ]",
    xlim=(-24.2, -19.5),
    ylim=(1e-11, 5e-3),
)
ax.text(0.05, 0.95, f"z ~ {z}", transform=ax.transAxes, va="top", ha="left")

plt.tight_layout()

fig.savefig(f"figures/lf_z={z}_meanZ.png", dpi=500)

Now errors from assuming no corrections

In [ ]:
Mobs_Kcorr = rng.normal(Mtrue, 0.2)

# bin the observed magnitudes
weights_Kcorr, _ = np.histogram(Mobs_Kcorr, bins=bins)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 3.3), dpi=120)

# plot the LF
ax.plot(M, sch_lf(M, sch_params[z]), c="gray", label="Luminosity function")

# plot the observed LF
binned_lf = weights_Kcorr / obs_vol / bin_width
errs = np.sqrt(weights_Kcorr) / obs_vol / bin_width
ax.errorbar(
    bin_centers, 
    binned_lf, 
    yerr=errs, 
    ls="", 
    marker=".", 
    markersize=4,
    capsize=3, 
    c="C3",
    label="Inferred using $K_{\mathrm{Corr}}=0$"
)

ax.legend(loc="lower right", fontsize=9)
ax.set(
    yscale="log", 
    xlabel="$M_{UV}$", 
    ylabel="$\phi(M_{UV})$  [Mpc$^{-3}$ mag$^{-1}$ ]",
    xlim=(-24.2, -19.5),
    ylim=(1e-11, 5e-3),
)
ax.text(0.05, 0.95, f"z ~ {z}", transform=ax.transAxes, va="top", ha="left")

plt.tight_layout()

fig.savefig(f"figures/lf_z={z}_Kcorr.png", dpi=500)

I will use the same distribution to explain Eddington Bias

In [ ]:
Mobs_eddington = rng.normal(Mtrue, 0.20)

# bin the observed magnitudes
weights_eddington, _ = np.histogram(Mobs_eddington, bins=bins)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 3.3), dpi=120)

# plot the LF
ax.plot(M, sch_lf(M, sch_params[z]), c="gray", label="Luminosity function")

# plot the observed LF
binned_lf = weights_eddington / obs_vol / bin_width
errs = np.sqrt(weights_eddington) / obs_vol / bin_width
ax.errorbar(
    bin_centers, 
    binned_lf, 
    yerr=errs, 
    ls="", 
    marker=".", 
    markersize=4,
    capsize=3, 
    c="C3",
    label="$M_{\mathrm{obs}} \sim \mathcal{N}(M_{\mathrm{true}}, 0.20)$"
)

ax.legend(loc="lower right", fontsize=9)
ax.set(
    yscale="log", 
    xlabel="$M_{UV}$", 
    ylabel="$\phi(M_{UV})$  [Mpc$^{-3}$ mag$^{-1}$ ]",
    xlim=(-24.2, -19.5),
    ylim=(1e-11, 5e-3),
)
ax.text(0.05, 0.95, f"z ~ {z}", transform=ax.transAxes, va="top", ha="left")

plt.tight_layout()

fig.savefig(f"figures/lf_z={z}_eddington.png", dpi=500)

Finally, I will simulate lensing

In [ ]:
def dPdmu(mu):
    dPm1dmu = 2 / (mu - 1)**3 * (mu > 2)
    dPm2dmu = 2 / (mu + 1)**3
    dP = dPm1dmu + dPm2dmu
    dP /= dP.sum()
    return dP

In [ ]:
# calculate the lensing probability on a grid
mugrid = np.linspace(0, 1000, 10_000_000)
mugrid = mugrid[1:]
dP = dPdmu(mugrid)

# sample magnification for lensed galaxies
tau = 0.0041
N_lensed = int(tau * N_galaxies)
mu_samples = rng.choice(mugrid, N_lensed, p=dP)

# assemble list of magnifications, including delensing
mu = (1 - 4 * tau) / (1 - tau) * np.ones(N_galaxies)
mu[:N_lensed] = mu_samples
rng.shuffle(mu)

# calculate magnitude shifts
deltaM = -2.5 * np.log10(mu)

In [ ]:
Mobs3 = Mtrue + deltaM

# bin the observed magnitudes
weights3, _ = np.histogram(Mobs3, bins=bins, weights=1/mu)

Bright end excess without lensing

In [ ]:
fig, ax = plot_lf(4, data=True)
ax.set(xlim=(-25, -15.5), ylim=(1e-10, 1e-1))
fig.savefig("figures/lf_bright_excess.png", dpi=500)

In [ ]:
fig, ax = plot_lf(4, data=True)
ax.set(xlim=(-25, -15.5), ylim=(1e-10, 1e-1))

# plot the lensed LF
binned_lf = weights3 / obs_vol / bin_width
ax.plot(
    bin_centers, 
    binned_lf, 
    c="C1",
    label="Lensed Luminosity",
    zorder=0,
    alpha=0.85
)
ax.legend()
fig.savefig("figures/lf_bright_excess_lensed.png", dpi=500)

In [ ]:
fig, ax = plot_lf(4, data=True, lsstComm=False)

# plot the lensed LF
binned_lf = weights3 / obs_vol / bin_width
ax.plot(
    bin_centers, 
    binned_lf, 
    c="C1",
    label="Lensed Luminosity",
    zorder=0,
    alpha=0.85
)
ax.legend()

fig.savefig("figures/lf_before_commissioning.png", dpi=500)

In [ ]:
fig, ax = plot_lf(4, data=True, lsstComm=True)

# plot the lensed LF
binned_lf = weights3 / obs_vol / bin_width
ax.plot(
    bin_centers, 
    binned_lf, 
    c="C1",
    label="Lensed Luminosity",
    zorder=0,
    alpha=0.85
)
ax.legend()

fig.savefig("figures/lf_after_commissioning.png", dpi=500)